# AIE425 Intelligent Recommender Systems, Fall Semester 25/26
Assignment 1: Neighborhood CF & Clustering in CF
| Name | ID | Part |
| ------ | -----| -----|
| Youssef Husseiny | 222101943 | Section 3 |
| Habiba Ahmed | 222100471 | Section 2 |
| Seif Amr Abdelhafez | 222102312 | Section 1 |

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys


In [3]:
m = pd.read_csv('../../dataset/movies.csv')
r = pd.read_csv('../../dataset/ratings.csv')

In [4]:
m

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


In [5]:
r

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858
...,...,...,...,...
32000199,200948,79702,4.5,1294412589
32000200,200948,79796,1.0,1287216292
32000201,200948,80350,0.5,1294412671
32000202,200948,80463,3.5,1350423800


In [6]:
os.getcwd()

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\section3_clustering_based_cf\\part1_user_clustering_avg_ratings'

In [7]:
import os
import sys

utils_path = os.path.abspath(os.path.join(os.getcwd(), '../..', 'utils'))

In [8]:
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [9]:
print(f"Calculated utils_path: {utils_path}")
print(f"Is utils_path in sys.path: {utils_path in sys.path}")

Calculated utils_path: d:\My_Laptop\University\GU\AI Science\Year 4\Semester 1\Intelligent Recommender System\Assignments\Assignment1\utils
Is utils_path in sys.path: True


In [10]:
os.listdir(utils_path)

['AdjustedCosine_ItemSim.py',
 'Cosine_Sim.py',
 'KMeans_Clustering.py',
 'MeanCenteredSimilarityRS.py',
 'Pearson_ItemSim.py',
 'Pearson_Sim.py',
 '__init__.py',
 '__pycache__']

In [11]:
utils_path

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\utils'

In [12]:
print(f"Original Data: {r['userId'].nunique()} Users, {r['movieId'].nunique()} Movies")

Original Data: 200948 Users, 84432 Movies


In [13]:
def read_ids_from_file(filepath):
    """Reads IDs from a file."""
    if not os.path.exists(filepath):
        print(f"Warning: File not found {filepath}")
        return []
    with open(filepath, 'r') as f:
        content = f.read().replace('\n', ',') 
        return [int(float(x)) for x in content.split(',') if x.strip().replace('.','',1).isdigit()]

In [14]:
target_users_ids = read_ids_from_file('../../Results/target_users.txt')
target_items_ids = read_ids_from_file('../../Results/target_items.txt')

In [15]:
print(f"Loaded Target Users: {target_users_ids}")
print(f"Loaded Target Items: {target_items_ids}")

Loaded Target Users: [79060, 183483, 55243]
Loaded Target Items: [203146, 230591]


In [16]:
top_movies = r['movieId'].value_counts().head(1200).index.tolist()
target_user_movies = r[r['userId'].isin(target_users_ids)]['movieId'].unique().tolist()
movies_to_keep = list(set(top_movies) | set(target_items_ids) | set(target_user_movies))
r_filtered = r[r['movieId'].isin(movies_to_keep)]

In [17]:
top_users = r_filtered['userId'].value_counts().head(110000).index.tolist()
users_to_keep = list(set(top_users) | set(target_users_ids))

r_filtered = r_filtered[r_filtered['userId'].isin(users_to_keep)]

In [18]:
matrix_df = r_filtered.pivot_table(index='userId', columns='movieId', values='rating')
matrix_user_ids = matrix_df.index.tolist()
matrix_movie_ids = matrix_df.columns.tolist()
rating_matrix_np = matrix_df.values

In [19]:
print(f"Users: {r_filtered['userId'].nunique()} (criteria: >= 100,000)")
print(f"Products: {r_filtered['movieId'].nunique()} (criteria: >= 1,000)")
print(f"Ratings: {len(r_filtered)} (criteria: >= 1,000,000)")

Users: 110003 (criteria: >= 100,000)
Products: 1216 (criteria: >= 1,000)
Ratings: 17728444 (criteria: >= 1,000,000)


In [20]:
matrix_df = r_filtered.pivot_table(index='userId', columns='movieId', values='rating')
matrix_df

movieId,1,2,3,5,6,7,10,11,16,17,...,185029,187593,192803,195159,202429,202439,203146,204698,207313,208703
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3.5,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2.5,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200943,3.5,2.5,NaN,NaN,3.5,4.5,3.5,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200944,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
matrix_user_ids = matrix_df.index.tolist()    
matrix_movie_ids = matrix_df.columns.tolist() 
rating_matrix_np = matrix_df.values
rating_matrix_np

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, 3.5, nan, ..., nan, nan, nan],
       [2.5, 2. , nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [4. , nan, nan, ..., nan, nan, nan],
       [5. , 1.5, nan, ..., nan, nan, nan]])

In [22]:
rating_matrix_np.shape

(110003, 1216)

In [23]:
X = matrix_df.fillna(0).values
user_ids = matrix_df.index.tolist()

In [24]:
X.shape

(110003, 1216)

In [27]:
from KMeans_Clustering import KMeans_Custom

In [28]:
kmeans = KMeans_Custom(n_clusters=3, max_iter=50, random_state=42)
kmeans.fit(X)

KeyboardInterrupt: 

In [ ]:
kmeans.inertia_

In [ ]:
kmeans.labels[:10]

In [ ]:
unique, counts = np.unique(kmeans.labels, return_counts=True)
for k, count in zip(unique, counts):
    print(f"Cluster {k}: {count} users")